## Chat GPT 4 Build

In [40]:
from collections import Counter, defaultdict

# Load the content of the provided text files
with open("five_letter_words.txt", "r") as file:
    five_letter_words = [word.strip().upper() for word in file.readlines()]

with open("OldWords.txt", "r") as file:
    OldWords = [word.strip().upper() for word in file.readlines()]

def remove_words(five_letter_words, OldWords):
    """Removes all words from `five_letter_words` that are also in `OldWords`."""
    return [word for word in five_letter_words if word not in OldWords]

five_letter_words = remove_words(five_letter_words, OldWords)

def count_letter_frequency(words_list):
    """Counts the frequency of each letter in the given list of words."""
    letter_frequency = Counter("".join(words_list))
    return sorted(letter_frequency.items(), key=lambda x: x[1], reverse=True)

top_common_letters = count_letter_frequency(five_letter_words)

def filter_words_based_on_letters(words_list, top_common_letters, num_letters=5):
    """Filters the words based on the most common letters and extracts unique letters."""
    filtered_words = words_list.copy()
    for letter, _ in top_common_letters[:num_letters]:
        filtered_words = [word for word in filtered_words if letter in word]
    
    unique_letters = set("".join(filtered_words))
    return filtered_words, list(unique_letters)

filtered_words, unique_letters = filter_words_based_on_letters(five_letter_words, top_common_letters)

def most_common_combinations(words_list, starting_letters, top_n=5):
    """Finds the most common two-letter combinations in the given list of words."""
    two_letter_combinations = Counter()
    for word in words_list:
        for i in range(len(word)):
            for j in range(i+1, len(word)):
                if word[i] in starting_letters and word[j] in starting_letters:
                    combo = tuple(sorted([word[i], word[j]]))
                    two_letter_combinations[combo] += 1
    return two_letter_combinations.most_common(top_n)


def filter_words_by_locked_positions(words_list, locked_positions):
    """Filters words that have the locked letters in the locked positions."""
    for position, letter in locked_positions.items():
        words_list = [word for word in words_list if len(word) >= position and word[position-1] == letter]
    return words_list

def filter_words_by_excluded_positions(words_list, excluded_positions):
    """Filters out words that have the specified letters in the excluded positions."""
    for position, letter in excluded_positions.items():
        words_list = [word for word in words_list if len(word) < position or word[position-1] != letter]
    return words_list


def filter_words_based_on_parameters_updated(words_list, letters_included, letters_not_included, locked_positions, excluded_positions):
    """Filters the word list based on the specified parameters."""
    # Filter by letters included
    for letter in letters_included:
        words_list = exclude_words_with_letters(words_list, letters_not_included)
    
    # Filter by letters not included
    words_list = exclude_words_with_letters(words_list, letters_not_included)
    
    # Filter by locked positions
    words_list = filter_words_by_locked_positions(words_list, locked_positions)
    
    # Filter by excluded positions
    words_list = filter_words_by_excluded_positions(words_list, excluded_positions)
    
    return words_list

# Renaming and defining the function for excluding words with certain letters
def exclude_words_with_letters(words_list, excluded_letters):
    """Excludes words that contain any of the letters marked as incorrect by the user."""
    for letter in excluded_letters:
        words_list = [word for word in words_list if letter not in word]
    return words_list

def exclude_words_with_letters(words_list, excluded_letters):
    """Excludes words that contain any of the letters marked as incorrect by the user."""
    for letter in excluded_letters:
        words_list = [word for word in words_list if letter not in word]
    return words_list


## Recomendations for next word

In [41]:
def compute_letter_scores(words_list):
    """Computes the frequency scores for each letter in the words list."""
    letter_frequency = Counter("".join(words_list))
    total_letters = sum(letter_frequency.values())
    
    # Normalize the letter scores
    letter_scores = {letter: freq / total_letters for letter, freq in letter_frequency.items()}
    return letter_scores

def compute_combination_scores(words_list, unique_letters):
    """Computes the scores for two-letter combinations in the words list."""
    two_letter_combinations = most_common_combinations(words_list, unique_letters, top_n=None)
    total_combinations = sum([count for _, count in two_letter_combinations])
    
    # Normalize the combination scores
    combination_scores = {combo: count / total_combinations for combo, count in two_letter_combinations}
    return combination_scores

def top_n_recommended_words_with_scores(words_list, top_common_letters, unique_letters, n=20, repeat_multiplier=0.5):
    """Recommends the top n optimal words based on letter and combination scores and returns the breakdown."""
    letter_scores = compute_letter_scores(words_list)
    combination_scores = compute_combination_scores(words_list, unique_letters)
    
    word_scores = {}
    word_letter_scores = {}
    word_combination_scores = {}
    for word in words_list:
        # Letter score
        word_letter_score = sum([letter_scores.get(letter, 0) for letter in word])
        word_letter_scores[word] = round(word_letter_score, 3)
        
        # Combination score
        word_combination_score = 0
        for i in range(len(word)):
            for j in range(i+1, len(word)):
                combo = tuple(sorted([word[i], word[j]]))
                word_combination_score += combination_scores.get(combo, 0)
        word_combination_scores[word] = round(word_combination_score, 3)
        
        # Apply the multiplier if the word has repeated letters
        multiplier = repeat_multiplier if len(word) > len(set(word)) else 1.0
                
        # Total score
        word_scores[word] = round((word_letter_score + word_combination_score) * multiplier, 3)
    
    # Get the top n words with the highest scores
    sorted_words = sorted(word_scores, key=word_scores.get, reverse=True)[:n]
    headers = ["Word", "Total Score", "Letter Score", "Combination Score"]
    return headers, sorted_words, [word_scores[word] for word in sorted_words], [word_letter_scores[word] for word in sorted_words], [word_combination_scores[word] for word in sorted_words]


In [42]:
def solve_wordle():
    # Fetch user's choices from the global variables
    global letters_included, letters_not_included, locked_positions, excluded_positions
    
    # Filter the words based on the specified parameters using the updated function
    filtered_words_updated = filter_words_based_on_parameters_updated(five_letter_words, letters_included, letters_not_included, locked_positions, excluded_positions)

    # Recommend the next word
    top_words, _, _, _ = top_n_recommended_words_with_scores(filtered_words_updated, top_common_letters, unique_letters, n=1)
    
    # Set the recommended word in the output label
    if top_words:
        output_var.set(f"Recommended word: {top_words[0]}")
    else:
        output_var.set("No recommended word found.")


## Graphical interphase

## Play the game

In [43]:
# Set the variables
letters_not_included = ['A', 'R', 'O', 'E', 'U', 'T', 'C', 'K'] #, 'W', 'N', 'T', 'M']
letters_included = ['N', 'I','S']
locked_positions = { 1: 'S', 2: 'N', 3: 'I' }
excluded_positions = {4: 'S', 5: 'S'}#{1: 'D', 5: 'P'}

# Set the repeat_multiplier
repeat_multiplier_value = 0.5

# Filter the words based on the specified parameters using the updated function
filtered_words_updated = filter_words_based_on_parameters_updated(five_letter_words, letters_included, letters_not_included, locked_positions, excluded_positions)

# Recommend the next word
headers, top_words, top_total_scores, top_letter_scores, top_combination_scores = top_n_recommended_words_with_scores(filtered_words_updated, top_common_letters, unique_letters, repeat_multiplier=repeat_multiplier_value)

# Display the results
print("Filtered words (first 10):", filtered_words_updated[:10])
print("Recommended next word:")
print(headers)
list(zip(top_words, top_total_scores, top_letter_scores, top_combination_scores))


Filtered words (first 10): ['SNIFF', 'SNIPY']
Recommended next word:
['Word', 'Total Score', 'Letter Score', 'Combination Score']


[('SNIPY', 0.8, 0.8, 0), ('SNIFF', 0.5, 1.0, 0)]

In [44]:
import tkinter as tk
from tkinter import StringVar, ttk
from functools import partial

# Initialize the main window
root = tk.Tk()
root.title("Wordle Solver")

# Variables to store user's choices
letters_included = []
letters_not_included = []
locked_positions = {}
excluded_positions = {}

# Command functions for buttons
def create_lock_command(button, letter, position):
    def command():
        lock_in_place(button, letter, position)
    return command

def create_exclude_command(button, letter):
    def command():
        do_not_include(button, letter)
    return command

def create_wrong_place_command(button, letter, position):
    def command():
        wrong_place(button, letter, position)
    return command

def lock_in_place(button, letter, position):
    global locked_positions  # Declare the variable as global
    locked_positions[position] = letter
    button.config(state=tk.DISABLED)

def do_not_include(button, letter):
    global letters_not_included  # Declare the variable as global
    letters_not_included.append(letter)
    button.config(state=tk.DISABLED)

def wrong_place(button, letter, position):
    global letters_not_included, excluded_positions  # Declare the variables as global
    excluded_positions[position] = letter
    letters_not_included.append(letter)
    button.config(state=tk.DISABLED)

    
from functools import partial

def display_letters():
    for widget in root.winfo_children():
        widget.grid_forget()

    word = word_entry.get().upper()
    for i, letter in enumerate(word):
        tk.Label(root, text=letter, width=5).grid(row=1, column=i+1)
        
        lock_button = tk.Button(root, text="Lock")
        lock_button['command'] = partial(lock_in_place, lock_button, letter, i+1)
        lock_button.grid(row=2, column=i+1)

        exclude_button = tk.Button(root, text="Exclude")
        exclude_button['command'] = partial(do_not_include, exclude_button, letter)
        exclude_button.grid(row=3, column=i+1)

        wrong_place_button = tk.Button(root, text="Wrong Place")
        wrong_place_button['command'] = partial(wrong_place, wrong_place_button, letter, i+1)
        wrong_place_button.grid(row=4, column=i+1)

    submit_btn.grid(row=0, column=2, padx=10, pady=10)
    solve_btn.grid(row=5, column=1, columnspan=5, pady=10)
    output_label.grid(row=6, column=1, columnspan=5, pady=10)



def solve_wordle():
    global letters_included, letters_not_included, locked_positions, excluded_positions
    
    # Filter the words based on the specified parameters using the updated function
    filtered_words_updated = filter_words_based_on_parameters_updated(five_letter_words, letters_included, letters_not_included, locked_positions, excluded_positions)
    
    # Recommend the next word
    top_words, top_total_scores, top_letter_scores, top_combination_scores, _ = top_n_recommended_words_with_scores(filtered_words_updated, top_common_letters, unique_letters)
    
    # Display the result (recommended word) in the GUI
    if top_words:
        output_var.set(f"Suggested word: {top_words[0]}")
    else:
        output_var.set("No word found!")


# Entry field for the 5-letter word
word_entry = tk.Entry(root, width=15)
word_entry.grid(row=0, column=1, padx=10, pady=10)

submit_btn = tk.Button(root, text="Submit Word", command=display_letters)
submit_btn.grid(row=0, column=2, padx=10, pady=10)

# Button to solve Wordle
solve_btn = tk.Button(root, text="Solve Wordle", command=solve_wordle)

solve_btn.grid(row=5, column=1, columnspan=5, pady=10)

# Output field to display the suggested word
output_var = StringVar()
output_var.set("Suggested word will appear here")
output_label = tk.Label(root, textvariable=output_var, width=25)
output_label.grid(row=6, column=1, columnspan=5, pady=10)

root.mainloop()
